## Import

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')
plt.rcParams.update({'font.size': 32})
plt.rcParams["figure.figsize"] = (12,8)
import torch
import numpy as np
import scipy.signal
import scipy.io
import pandas as pd
import itertools
from itertools import product

from tqdm.notebook import tqdm, trange

In [ ]:
from google.colab import files

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import random

In [ ]:
import h5py

Data import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fs = 50e3

In [ ]:
dataset = h5py.File('/content/drive/MyDrive/dataset_.hdf5', 'r')

In [ ]:
class H5Dataset:
    
    def __init__(self, h5dataset, segment_size = 1):
        self.dataset  = h5dataset
        self.ds_parts = list(self.dataset.keys())
        self.seg_size = int(segment_size)
        
    def visit(self):
        self.dataset.visit(lambda name: print(name))
    
    def inspect(self):
        for key in self.ds_parts:
            part = self.dataset[key]
            info = f'Name {key},\tType {type(part)}'
            info +=f' Shape {part.shape}, DType {part.dtype}'
            if isinstance(self.seg_size,int):
                info +=f' N segments {part.shape[1]//self.seg_size}'    
            print(info)
    
    def __len__(self):
        return self.length
    
    def get_source(self, part, source, label = None):
        if self.seg_size is None: raise ValueError
        
        source_data = self.dataset[part][source]
        
        segment = source_data.reshape(-1,self.seg_size)
        
        if label is None:
            label = source
        target  = label * np.ones(segment.shape[0])
        return segment, target

    def part(self, part_name):
        return self.dataset[part_name]
    
#     def get_batches_idxs 
    
    def get_range(self, part, sources = None, segments = None, label = None):
        if self.seg_size is None: raise ValueError
        
        data = self.dataset[part]
        if sources is None:  sources  = (0,data.shape[0])
        if segments is None: segments = (0,data.shape[1]//self.seg_size) 
        sources  = (max(0,int(sources[0])), min(data.shape[0],int(sources[1])))    
        segments = (segments[0]*self.seg_size,segments[1]*self.seg_size)  
        segments = (max(0,int(segments[0])),min(data.shape[1],int(segments[1])))

        return data[sources[0]:sources[1],
                    segments[0]:segments[1] ]
        
    

In [ ]:
SIZE_SIGNAL = 10e3
SIZE_SIGNAL = int(SIZE_SIGNAL)

ds = H5Dataset(dataset, segment_size=int(SIZE_SIGNAL))
ds.inspect()

Name x_test_1,	Type <class 'h5py._hl.dataset.Dataset'> Shape (20, 12500000), DType float32 N segments 1250
Name x_test_2,	Type <class 'h5py._hl.dataset.Dataset'> Shape (20, 12500000), DType float32 N segments 1250
Name x_train_1,	Type <class 'h5py._hl.dataset.Dataset'> Shape (20, 12500000), DType float32 N segments 1250
Name x_train_2,	Type <class 'h5py._hl.dataset.Dataset'> Shape (20, 12500000), DType float32 N segments 1250


In [ ]:
range_ = ds.get_range('x_test_1')
print(range_.shape)

(20, 12500000)


In [ ]:
range_ = ds.get_range('x_test_1',(0,2))
print(range_.shape)

(2, 12500000)


we have period about 1000 points

In [ ]:
# !nvidia-smi --gpu-reset 
# import gc
# gc.collect()

In [ ]:
# Cell
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import itertools

# Cell
class MiniRocketFeaturesPlus(nn.Module):
    fitting = False

    def __init__(self, c_in, seq_len, num_features=10_000, max_dilations_per_kernel=32, kernel_size=9, max_num_channels=1, max_num_kernels=84,
                 add_lsaz=False):
        super(MiniRocketFeaturesPlus, self).__init__()
        self.c_in, self.seq_len = c_in, seq_len
        self.kernel_size, self.max_num_channels, self.add_lsaz = kernel_size, max_num_channels, add_lsaz

        # Kernels
        indices, pos_values = self.get_indices(kernel_size, max_num_kernels)
        self.num_kernels = len(indices)
        kernels = (-torch.ones(self.num_kernels, 1, self.kernel_size)).scatter_(2, indices, pos_values).to(device)
        self.indices = indices
        self.kernels = nn.Parameter(kernels.repeat(c_in, 1, 1), requires_grad=False)
        if add_lsaz:
            num_features = num_features // 2
        self.num_features = num_features // self.num_kernels * self.num_kernels
        self.max_dilations_per_kernel = max_dilations_per_kernel

        # Dilations
        self.set_dilations(seq_len)

        # Channel combinations (multivariate)
        if c_in > 1:
            self.set_channel_combinations(c_in, max_num_channels)

        # Bias
        for i in range(self.num_dilations):
            self.register_buffer(f'biases_{i}', torch.empty(
                (self.num_kernels, self.num_features_per_dilation[i])))
        self.register_buffer('prefit', torch.BoolTensor([False]))

    def forward(self, x):
        _features = []
        for i, (dilation, padding) in enumerate(zip(self.dilations, self.padding)):
            _padding1 = i % 2

            # Convolution
            C = F.conv1d(x, self.kernels.to(device), padding=padding,
                        dilation=dilation, groups=self.c_in)
            if self.c_in > 1:  # multivariate
                C = C.reshape(x.shape[0], self.c_in, self.num_kernels, -1)
                channel_combination = getattr(
                    self, f'channel_combinations_{i}')
                C = torch.mul(C, channel_combination)
                C = C.sum(1)

            # Bias
            if not self.prefit or self.fitting:
                num_features_this_dilation = self.num_features_per_dilation[i]
                bias_this_dilation = self.get_bias(
                    C, num_features_this_dilation)
                setattr(self, f'biases_{i}', bias_this_dilation)
                if self.fitting:
                    if i < self.num_dilations - 1:
                        continue
                    else:
                        self.prefit = torch.BoolTensor([True])
                        return
                elif i == self.num_dilations - 1:
                    self.prefit = torch.BoolTensor([True])
            else:
                bias_this_dilation = getattr(self, f'biases_{i}')

            # Features
            _features.append(self.get_PPVs(
                C[:, _padding1::2], bias_this_dilation[_padding1::2]))
            _features.append(self.get_PPVs(
                C[:, 1-_padding1::2, padding:-padding], bias_this_dilation[1-_padding1::2]))

        return torch.cat(_features, dim=1)

    def fit(self, X, chunksize=None):
        num_samples = X.shape[0]
        if chunksize is None:
            chunksize = min(num_samples, self.num_dilations * self.num_kernels)
        else:
            chunksize = min(num_samples, chunksize)
        idxs = np.random.choice(num_samples, chunksize, False)
        self.fitting = True
        if isinstance(X, np.ndarray):
            self(torch.from_numpy(X[idxs]).to(self.kernels.device))
        else:
            self(X[idxs].to(self.kernels.device))
        self.fitting = False

    def get_PPVs(self, C, bias):
        C = C.unsqueeze(-1)
        bias = bias.view(1, bias.shape[0], 1, bias.shape[1])
        a = (C > bias).float().mean(2).flatten(1)
        if self.add_lsaz:
            dif = (C - bias)
            b = (F.relu(dif).sum(2) /
                 torch.clamp_min(torch.abs(dif).sum(2), 1e-8)).flatten(1)
            return torch.cat((a, b), dim=1)
        else:
            return a

    def set_dilations(self, input_length):
        num_features_per_kernel = self.num_features // self.num_kernels
        true_max_dilations_per_kernel = min(
            num_features_per_kernel, self.max_dilations_per_kernel)
        multiplier = num_features_per_kernel / true_max_dilations_per_kernel
        max_exponent = np.log2((input_length - 1) / (self.kernel_size - 1))
        dilations, num_features_per_dilation = \
            np.unique(np.logspace(0, max_exponent, true_max_dilations_per_kernel, base=2).astype(
                np.int32), return_counts=True)
        num_features_per_dilation = (
            num_features_per_dilation * multiplier).astype(np.int32)
        remainder = num_features_per_kernel - num_features_per_dilation.sum()
        i = 0
        while remainder > 0:
            num_features_per_dilation[i] += 1
            remainder -= 1
            i = (i + 1) % len(num_features_per_dilation)
        self.num_features_per_dilation = num_features_per_dilation
        self.num_dilations = len(dilations)
        self.dilations = dilations
        self.padding = []
        for i, dilation in enumerate(dilations):
            self.padding.append((((self.kernel_size - 1) * dilation) // 2))

    def set_channel_combinations(self, num_channels, max_num_channels):
        num_combinations = self.num_kernels * self.num_dilations
        if max_num_channels:
            max_num_channels = min(num_channels, max_num_channels)
        else:
            max_num_channels = num_channels
        max_exponent_channels = np.log2(max_num_channels + 1)
        num_channels_per_combination = (
            2 ** np.random.uniform(0, max_exponent_channels, num_combinations)).astype(np.int32)
        self.num_channels_per_combination = num_channels_per_combination
        channel_combinations = torch.zeros(
            (1, num_channels, num_combinations, 1))
        for i in range(num_combinations):
            channel_combinations[:, np.random.choice(
                num_channels, num_channels_per_combination[i], False), i] = 1
        channel_combinations = torch.split(
            channel_combinations, self.num_kernels, 2)  # split by dilation
        for i, channel_combination in enumerate(channel_combinations):
            self.register_buffer(
                f'channel_combinations_{i}', channel_combination)  # per dilation

    def get_quantiles(self, n):
        return torch.tensor([(_ * ((np.sqrt(5) + 1) / 2)) % 1 for _ in range(1, n + 1)]).float()

    def get_bias(self, C, num_features_this_dilation):
        isp = torch.randint(C.shape[0], (self.num_kernels,))
        samples = C[isp].diagonal().T
        biases = torch.quantile(samples, self.get_quantiles(
            num_features_this_dilation).to(C.device), dim=1).T
        return biases

    def get_indices(self, kernel_size, max_num_kernels):
        num_pos_values = math.ceil(kernel_size / 3)
        num_neg_values = kernel_size - num_pos_values
        pos_values = num_neg_values / num_pos_values
        if kernel_size > 9:
            random_kernels = [np.sort(np.random.choice(kernel_size, num_pos_values, False)).reshape(
                1, -1) for _ in range(max_num_kernels)]
            indices = torch.from_numpy(
                np.concatenate(random_kernels, 0)).unsqueeze(1)
        else:
            indices = torch.LongTensor(list(itertools.combinations(
                np.arange(kernel_size), num_pos_values))).unsqueeze(1)
            if max_num_kernels and len(indices) > max_num_kernels:
                indices = indices[np.sort(np.random.choice(
                    len(indices), max_num_kernels, False))]
        return indices, pos_values

In [ ]:
minirockert = MiniRocketFeaturesPlus(c_in = 1, seq_len= SIZE_SIGNAL, num_features=2000, max_dilations_per_kernel=32, max_num_kernels=84)

In [ ]:
minirockert.dilations, minirockert.dilations.shape

(array([   1,    2,    3,    5,    6,    9,   13,   18,   25,   35,   48,
          67,   93,  129,  178,  247,  341,  472,  653,  903, 1249],
       dtype=int32), (21,))

In [ ]:
minirockert.seq_len

10000

In [ ]:
max_exponent = np.log2((minirockert.seq_len - 1) / (9 - 1))

num_features_per_kernel = minirockert.num_features // minirockert.num_kernels

true_max_dilations_per_kernel = min(
            num_features_per_kernel, minirockert.max_dilations_per_kernel)


dilations, num_features_per_dilation = np.unique(np.logspace(0, max_exponent, true_max_dilations_per_kernel, base = 2).astype(np.int32), return_counts = True)
    
    # num_features_per_dilation = (num_features_per_dilation * multiplier).astype(np.int32) # this is a vector

In [ ]:
num_features_per_dilation

array([3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
np.unique(np.logspace(0, max_exponent, true_max_dilations_per_kernel, base = 2)).shape

(23,)

In [ ]:
np.asarray(minirockert.padding), np.asarray(minirockert.padding).shape

(array([   4,    8,   12,   20,   24,   36,   52,   72,  100,  140,  192,
         268,  372,  516,  712,  988, 1364, 1888, 2612, 3612, 4996]), (21,))

In [ ]:
minirockert.kernels.data.cpu().numpy().squeeze()

array([[ 2.,  2.,  2., -1., -1., -1., -1., -1., -1.],
       [ 2.,  2., -1.,  2., -1., -1., -1., -1., -1.],
       [ 2.,  2., -1., -1.,  2., -1., -1., -1., -1.],
       [ 2.,  2., -1., -1., -1.,  2., -1., -1., -1.],
       [ 2.,  2., -1., -1., -1., -1.,  2., -1., -1.],
       [ 2.,  2., -1., -1., -1., -1., -1.,  2., -1.],
       [ 2.,  2., -1., -1., -1., -1., -1., -1.,  2.],
       [ 2., -1.,  2.,  2., -1., -1., -1., -1., -1.],
       [ 2., -1.,  2., -1.,  2., -1., -1., -1., -1.],
       [ 2., -1.,  2., -1., -1.,  2., -1., -1., -1.],
       [ 2., -1.,  2., -1., -1., -1.,  2., -1., -1.],
       [ 2., -1.,  2., -1., -1., -1., -1.,  2., -1.],
       [ 2., -1.,  2., -1., -1., -1., -1., -1.,  2.],
       [ 2., -1., -1.,  2.,  2., -1., -1., -1., -1.],
       [ 2., -1., -1.,  2., -1.,  2., -1., -1., -1.],
       [ 2., -1., -1.,  2., -1., -1.,  2., -1., -1.],
       [ 2., -1., -1.,  2., -1., -1., -1.,  2., -1.],
       [ 2., -1., -1.,  2., -1., -1., -1., -1.,  2.],
       [ 2., -1., -1., -1., 

In [ ]:
minirockert.kernels.data.cpu().numpy().squeeze().shape

(84, 9)

In [ ]:
minirockert.num_features

1932

In [ ]:
minirockert.num_dilations

21

In [ ]:
minirockert.num_kernels 

84

In [ ]:
kernels = pd.DataFrame()


for i, (dilation, padding) in enumerate(zip(minirockert.dilations, minirockert.padding)):
  bias_this_dilation = getattr(minirockert, f'biases_{i}')
  # print(np.array(bias_this_dilation.data.cpu().numpy().squeeze()))
  for kernel, bias in zip(minirockert.kernels.data.cpu().numpy().squeeze(),
                         bias_this_dilation.data.cpu().numpy().squeeze().astype(float)):
      if bias.ndim>0:
        bias_ = bias
        for bias in bias_:
          kernels = kernels.append({'kernel': kernel,
                              'bias': bias,
                              'dilation':dilation,
                              'padding':padding}, ignore_index=True)
      else:
        kernels = kernels.append({'kernel': kernel,
                'bias': bias,
                'dilation':dilation,
                'padding':padding}, ignore_index=True)
      
kernels.to_json('/content/drive/MyDrive/kernels.json', orient='records')
kernels = pd.read_json('/content/drive/MyDrive/kernels.json', orient='records')
files.download('/content/drive/MyDrive/kernels.json')
kernels.head(3)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,kernel,bias,dilation,padding
0,"[2.0, 2.0, 2.0, -1.0, -1.0, -1.0, -1.0, -1.0, ...",0.025633,1,4
1,"[2.0, 2.0, 2.0, -1.0, -1.0, -1.0, -1.0, -1.0, ...",0.000000,1,4
2,"[2.0, 2.0, 2.0, -1.0, -1.0, -1.0, -1.0, -1.0, ...",-0.023275,1,4


In [ ]:
torch.cuda.empty_cache()
x_ = torch.tensor(x_)
out = minirockert(x_.to(device).unsqueeze(1))

In [ ]:
CLASSES_PER_PART = 20
def get_features(ds, parts, fs):
    parts = sum([],parts)
    df = pd.DataFrame([])    
    
    for  (cntpart,part),i in tqdm( product(enumerate(parts), range(CLASSES_PER_PART)  ) ):

        x_,y_ = ds.get_source(part,i)
        
        y_ = (cntpart*CLASSES_PER_PART+i)*np.ones(x_.shape[0])
        
        df_ = pd.DataFrame(minirockert(torch.from_numpy(x_).to(device).unsqueeze(1)).data.cpu().numpy())
        
        df = pd.concat([df,df_ ],axis=0, ignore_index=True)
    
    return df


In [ ]:
df_train = get_features(ds, ['x_train_1','x_train_2'], fs = fs)

0it [00:00, ?it/s]

In [ ]:

df_train.to_csv('/content/drive/MyDrive/minirocket3000_train.csv', index=False)

In [ ]:
df_test = get_features(ds, ['x_test_1','x_test_2'], fs = fs)

0it [00:00, ?it/s]

In [ ]:
df_test.to_csv('/content/drive/MyDrive/minirocket3000_test.csv', index=False)

In [ ]:
df_train.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931
28028,0.4095,0.7449,0.1391,0.4289,0.7081,0.1704,0.4421,0.6760,0.2077,0.4261,...,1.000,0.00,1.000,1.000,0.000,0.000,1.00,0.00,0.00,1.000
32172,0.3898,0.7772,0.1182,0.4001,0.7524,0.1308,0.4099,0.7277,0.1510,0.3993,...,1.000,0.00,1.000,1.000,0.000,0.000,1.00,0.00,1.00,1.000
13170,0.3862,0.7775,0.1438,0.3788,0.7709,0.1453,0.3793,0.7691,0.1464,0.3796,...,0.000,0.00,0.000,0.375,0.000,0.000,0.00,0.00,0.25,0.000
13136,0.3841,0.7807,0.1431,0.3735,0.7798,0.1448,0.3729,0.7805,0.1491,0.3778,...,1.000,0.00,0.500,1.000,0.000,0.000,1.00,0.00,0.50,0.000
39357,0.4201,0.7258,0.1571,0.4340,0.6851,0.1922,0.4457,0.6555,0.2299,0.4289,...,0.000,0.25,0.000,0.000,0.000,1.000,0.00,0.25,0.75,0.000
33357,0.4453,0.6845,0.1858,0.4470,0.6607,0.2245,0.4571,0.6420,0.2591,0.4451,...,0.875,1.00,0.625,0.125,1.000,1.000,0.75,1.00,0.00,0.625
31259,0.3833,0.7921,0.1120,0.3944,0.7693,0.1256,0.4124,0.7325,0.1450,0.3908,...,0.500,1.00,0.000,0.000,0.375,0.375,0.00,0.50,0.25,0.000
18839,0.3742,0.7714,0.1519,0.3625,0.7742,0.1541,0.3735,0.7723,0.1581,0.3717,...,1.000,1.00,1.000,1.000,1.000,0.000,1.00,1.00,0.00,1.000
23742,0.3547,0.8113,0.1432,0.3541,0.8035,0.1423,0.3647,0.7884,0.1515,0.3360,...,0.000,0.00,0.000,0.000,0.000,1.000,1.00,1.00,1.00,1.000
18972,0.3640,0.7734,0.1511,0.3561,0.7774,0.1507,0.3663,0.7773,0.1539,0.3678,...,0.000,1.00,0.000,0.000,1.000,1.000,0.00,1.00,0.00,0.000


In [ ]:
df_test.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931
45630,0.4217,0.7179,0.1489,0.4293,0.6929,0.1796,0.4377,0.6717,0.2073,0.4259,...,1.0,0.0,0.875,1.0,0.125,0.00,1.000,0.0,0.0,1.000
10605,0.3939,0.7585,0.1532,0.3873,0.7560,0.1541,0.3859,0.7554,0.1587,0.3930,...,1.0,1.0,1.000,1.0,1.000,0.00,1.000,1.0,0.0,1.000
158,0.3865,0.7597,0.1439,0.3770,0.7604,0.1449,0.3780,0.7619,0.1455,0.3792,...,1.0,1.0,1.000,1.0,1.000,0.00,1.000,1.0,0.0,1.000
29115,0.4366,0.6994,0.1765,0.4492,0.6635,0.2074,0.4566,0.6352,0.2580,0.4374,...,1.0,0.0,1.000,1.0,1.000,0.00,1.000,0.0,0.0,1.000
32596,0.4358,0.6759,0.1931,0.4441,0.6480,0.2292,0.4560,0.6310,0.2626,0.4501,...,1.0,1.0,1.000,1.0,1.000,0.00,1.000,1.0,0.0,1.000
15005,0.3980,0.7540,0.1666,0.3926,0.7510,0.1690,0.3965,0.7456,0.1690,0.3961,...,0.0,0.0,0.000,0.0,0.000,0.00,0.000,0.0,0.0,0.000
5738,0.3860,0.7508,0.1443,0.3794,0.7496,0.1478,0.3837,0.7486,0.1479,0.3857,...,0.0,0.0,0.000,0.0,0.000,0.25,0.000,0.0,1.0,0.000
27205,0.4221,0.7466,0.1339,0.4249,0.7137,0.1663,0.4389,0.6806,0.2049,0.4326,...,1.0,1.0,1.000,1.0,1.000,0.50,1.000,1.0,0.0,1.000
41677,0.3996,0.7560,0.1262,0.4141,0.7211,0.1495,0.4312,0.6954,0.1788,0.4152,...,0.0,1.0,0.000,0.0,0.250,0.25,0.000,1.0,0.0,0.000
41909,0.4015,0.7582,0.1239,0.4153,0.7186,0.1489,0.4305,0.6939,0.1836,0.4159,...,1.0,1.0,1.000,1.0,1.000,0.25,0.875,1.0,0.0,0.875
